# Shunting Yard Algo

In [8]:
def shuntingYardAlgo(infix):
    postfix = ""
    stack = []
    classes_flag=False
    precedence = {'*': 5, '+': 4, '?': 3, '.': 2, '|': 1}
    alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    digits = '0123456789'
    opened=0

    for c in infix:
        if c=='(' or c=='[':
            stack.append(c)
            classes_flag=True if c=='[' else False
            opened+=1
        elif c==')' or c==']':
            while stack[-1]!='(' and stack[-1]!='[':
                postfix += stack.pop()
            if stack==[]:
                return False
            
            stack.pop()
            opened -= 1
            classes_flag=False
        elif c in precedence:
            while stack and precedence.get(stack[-1], 0) >= precedence[c]:
                postfix += stack.pop()
            stack.append(c)
        elif c=='-':
            if(c==infix[0] or c==infix[-1]):
                return False
            final=infix[infix.index(c)+1]
            first=postfix[-1]
            proceeded_list=list()
            for a in alphabet+digits:
                if a>=first and a<=final:
                    proceeded_list.append(a)    
                    proceeded_list.append('|')

            infix+= ''.join(proceeded_list[:-1])

        else:
            postfix += c
            if classes_flag:
                postfix += '|'
    if opened != 0:
        return False

    while stack:
        postfix += stack.pop()

    return postfix

    



# Postfix to NFA

In [9]:
class state :
    def __init__(self,label):
        self.label = label
        self.edges =[]
class edge :
    def __init__(self,value,destination):
        self.value = value
        self.destination = destination

class NFA:
    def __init__(self, initial, accept):
        self.initial = initial
        self.accept = accept


def MakeState(counter,val,stack):
    state1 = state("S"+str(counter))
    state2 = state("S"+str(counter+1))
    counter+=2
    state1.edges.append(edge(val,state2))
    nfa_made= NFA(state1,state2)
    stack.append(nfa_made)

def zeroOrMore(counter,state,stack):
    
    nfa_made=stack.pop()
    state1 = state("S"+str(counter))
    state2 = state("S"+str(counter+1))
    
    counter+=2
    
    state1.edges.append(edge('&',nfa_made.initial))
    state1.edges.append(edge('&',state2))
    nfa_made.accept.edges.append(edge('&',state2))
    nfa_made.accept.edges.append(edge('&',nfa_made.initial))
    nfa_final = NFA(state1,state2)
    stack.append(nfa_final)

def oneOrMore(counter,state,stack):
    nfa_made=stack.pop()
    state1 = state("S"+str(counter))
    state2 = state("S"+str(counter+1))
    
    counter+=2
    
    state1.edges.append(edge('&',nfa_made.initial))
    nfa_made.accept.edges.append(edge('&',state2))
    nfa_made.accept.edges.append(edge('&',nfa_made.initial))
    nfa_final = NFA(state1,state2)
    stack.append(nfa_final)

def zeroOrOne(counter,state,stack):
    nfa_made=stack.pop()
    state1 = state("S"+str(counter))
    state2 = state("S"+str(counter+1))
    
    counter+=2
    
    state1.edges.append(edge('&',nfa_made.initial))
    state1.edges.append(edge('&',state2))
    nfa_made.accept.edges.append(edge('&',state2))
    nfa_final = NFA(state1,state2)
    stack.append(nfa_final)

def concatenate(counter,state,stack):
    nfa_made2=stack.pop()
    nfa_made1=stack.pop()
    nfa_made1.accept.edges.append(edge('&',nfa_made2.initial))
    nfa_final = NFA(nfa_made1.initial,nfa_made2.accept)
    stack.append(nfa_final)

def oring(counter,state,stack):
    nfa_made2=stack.pop()
    nfa_made1=stack.pop()
    state1 = state("S"+str(counter))
    state2 = state("S"+str(counter+1))
    counter+=2
    state1.edges.append(edge('&',nfa_made1.initial))
    state1.edges.append(edge('&',nfa_made2.initial))
    nfa_made1.accept.edges.append(edge('&',state2))
    nfa_made2.accept.edges.append(edge('&',state2))
    nfa_final = NFA(state1,state2)
    stack.append(nfa_final)


def thompsons(postfix):
    stack = []
    counter = 0
    alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    digits = '0123456789'
    for c in postfix:
        if c in alphabet or c in digits:
            MakeState(counter,c,stack)
            counter+=2
        elif c == '*':
            zeroOrMore(counter,c,stack)
            counter+=2
        elif c == '+':
            oneOrMore(counter,c,stack)
            counter+=2
        elif c == '?':
            zeroOrOne(counter,c,stack)
            counter+=2
        elif c == '.':
            concatenate(counter,c,stack)
        elif c == '|':
            oring(counter,c,stack)
    return stack.pop()





